In [1]:
from diffUV import dyned_eul, dyn_body
from diffUV.utils.symbol import *
from casadi import Function
from blue_rov import Params as ps
import numpy as np

In [2]:
r_g = vertcat(x_g, y_g, z_g) # center of gravity wrt body origin
r_b = vertcat(x_b, y_b, z_b) # center of buoyancy wrt body origin 
I_o = vertcat(I_x, I_y, I_z,I_xz) # rigid body inertia wrt body origin

added_m = vertcat(X_du, Y_dv, Z_dw, K_dp, M_dq, N_dr) # added mass in diagonals
coupling_added_m =  vertcat(X_dq, Y_dp, N_dp, M_du, K_dv) # effective added mass in non diagonals

linear_dc = vertcat(X_u, Y_v, Z_w, K_p,  M_q, N_r) # linear damping coefficients
quadratic_dc = vertcat(X_uu, Y_vv, Z_ww, K_pp, M_qq, N_rr) # quadratic damping coefficients

n0 = vertcat(dx, dy, dz, thet, phi, psi, dphi, dthet, dpsi) # state variables wrt ned
v0 = x_nb # velocity state variables wrt body

<h1>System Inertia Matrix example</h1>

In [3]:
# body representaion
uv_body = dyn_body()
inertia_mat = uv_body.body_inertia_matrix()
M_func = Function('M_b', [m, I_o, z_g, added_m, coupling_added_m], [inertia_mat]) # for numerical & symbolic use

In [4]:
# example usage of inertia mat
M_func(ps.m, ps.Io, ps.rg[2], ps.added_m, ps.coupl_added_m)

DM(
[[17, 0, 0, 0, 0.23, 0], 
 [0, 24.2, 0, -0.23, 0, 0], 
 [0, 0, 26.07, 0, 0, 0], 
 [0, -0.23, 0, 0.28, 0, 0], 
 [0.23, 0, 0, 0, 0.28, 0], 
 [0, 0, 0, -0, 0, 0.28]])

In [5]:
# ned representaion
uv_ned = dyned_eul()
inertia_ned_mat = uv_ned.ned_euler_inertia_matrix()
Onb = vertcat(thet, phi, psi)
M_ned_func = Function('M_ned', [m, I_o, z_g, Onb, added_m, coupling_added_m], [inertia_ned_mat]) # for numerical & symbolic use

In [7]:
# example usage of inertia mat in ned
Onb_v = np.array([0,0,0])
M_ned_func(ps.m, ps.Io, ps.rg[2], Onb_v,  ps.added_m, ps.coupl_added_m)

DM(
[[17, 0, 0, -0, 0.23, 0], 
 [0, 50.27, -26.07, -0.23, 0, 0], 
 [0, -26.07, 26.07, -0, -0, 0], 
 [-0, -0.23, -0, 0.28, 0, -0], 
 [0.23, 0, -0, 0, 0.28, 0], 
 [0, 0, 0, -0, 0, 0.28]])

<h1>Forward dynamics simulation example</h1>

In [14]:
uv_ned = dyned_eul()
forward_dynamics = uv_ned.ned_euler_forward_dynamics()
# forward_dynamics.size()

fd_func = Function('f', [m, W, B, r_g, r_b, I_o, 
                   added_m, coupling_added_m, 
                   linear_dc, quadratic_dc,
                   v0, n0, tau_b], [forward_dynamics])

In [16]:
# example usage of forward dynamics
tau = np.array([1,0,0,0,0,0])
v0_v = np.array([0, 0, 0 ,0 ,0, 0])
n0_v = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])
fd_func(ps.m, ps.W, ps.B, ps.rg, ps.rb, ps.Io, ps.added_m, ps.coupl_added_m, ps.linear_dc,ps.quadratic_dc,v0_v,  n0_v,  tau)

DM([0.0594846, 4.16334e-17, -0.0767165, 2.34535e-15, -0.0488624, 0])